# HW2-1: Where/How to apply NN?

## 레시피 주제 블로그 문서의 서브토픽 자동 태깅
레시피 주제의 블로그 문서를 서브토픽에 따라 검색결과를 보여주는 과제를 진행중입니다.
예를 들어, 가리비의 서브토픽 가운데,'가리비 손질' 이라는 것이 있다면, 여러 문서들
가운데, '가리비 손질'이라는 키워드가 들어간 문서만 보여주는 것입니다. 
이때, 검색결과의 품질을 높이려면, 키워드 뿐만 아니라, '손질' 이라는 의미에 해당하는
문맥정보가 문서에 적절히 들어 있어야 할 것입니다.
지식백과에서 '가리비 손질법'을 찾아보면 아래와 같이 나타납니다.
> 보관법 : 신선도를 유지하는 것이 가장 중요하며 냉장고에 보관하고 되도록이면 빨리 먹도록 한다.

NN을 써서 해보고자하는 것은 임의의 문장/문서가 들어왔을 때, 이것이 해당 서브토픽을 포함하는지
여부를 분류하는 것입니다. 
* 학습셋: 위와 같은 지식백과 추출 문장을 정답으로 간주하 데이터 생성
* 모델: 워드임베딩을 포함한 RNN, LSTM (?)

# HW2-2: Predicting imdb score

## Dataset: movie_metadata.csv
* x_data: numerical features (e.g., num_ciritic_for_reviews, director_face_book_likes, ..., total: 15)
* y_data: imdb_score (2 classes)
  * 0~9 --> 1 ( > 5) / 0 ( <= 5)

## Model
 * FNN 
  * \# of hidden layers: 3, 5
  * with drop out / without drop out 
 * hyperparameter
   * learning rate: 0.01
   * epochs: 10000
   
## Results
 * 3 hidden layers + no drop out: 79.4%
 * 3 hidden layers + drop out: 79.9%
 * 5 hidden layers + no drop out: 79.1%
 * 5 hidden layers + drop out: 70.1%

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA # Principal Component Analysis module
from sklearn.cluster import KMeans # KMeans clustering 
import matplotlib.pyplot as plt # Python defacto plotting library
%matplotlib inline 
import tensorflow as tf

In [2]:
movie = pd.read_csv('./movie_metadata.csv') # reads the csv and creates the dataframe called movie
movie.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [3]:
str_list = [] # empty list to contain columns with strings (words)
for colname, colvalue in movie.iteritems():
    if type(colvalue[1]) == str:
         str_list.append(colname)
# Get to the numeric columns by inversion            
num_list = movie.columns.difference(str_list)  

In [4]:
movie_num = movie[num_list]
#del movie # Get rid of movie df as we won't need it now
movie_num = movie_num.fillna(value=0, axis=1)
movie_num.head()
movie_num.keys()

Index([u'actor_1_facebook_likes', u'actor_2_facebook_likes',
       u'actor_3_facebook_likes', u'aspect_ratio', u'budget',
       u'cast_total_facebook_likes', u'director_facebook_likes', u'duration',
       u'facenumber_in_poster', u'gross', u'imdb_score',
       u'movie_facebook_likes', u'num_critic_for_reviews',
       u'num_user_for_reviews', u'num_voted_users', u'title_year'],
      dtype='object')

In [5]:
feature_list = list(movie_num.keys())
feature_list.remove('imdb_score')

print feature_list, len(feature_list)
norm_array = [] 
for f in feature_list:
    norm_array.append(movie_num[f])
    
X = np.dstack(norm_array)
print len(X[0])
X = np.reshape(X, (len(X[0]),15))
print X.shape

# Data Normalization
from sklearn.preprocessing import StandardScaler
x_data = X_std = StandardScaler().fit_transform(X)
y_data = np.array([ [int(score)] for score in movie_num['imdb_score']])
for i, score in enumerate(movie_num['imdb_score']):
    if int(score) > 5:
        y_data[i][0] = 1
    else:
        y_data[i][0] = 0
print y_data.shape
print max(y_data)
print min(y_data)

['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 'aspect_ratio', 'budget', 'cast_total_facebook_likes', 'director_facebook_likes', 'duration', 'facenumber_in_poster', 'gross', 'movie_facebook_likes', 'num_critic_for_reviews', 'num_user_for_reviews', 'num_voted_users', 'title_year'] 15
5043
(5043, 15)
(5043, 1)
[1]
[0]


In [6]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

print x_data.shape, x_train.shape, x_test.shape
print y_data.shape, y_train.shape, y_test.shape

(5043, 15) (4034, 15) (1009, 15)
(5043, 1) (4034, 1) (1009, 1)


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.01


In [8]:
X = tf.placeholder(tf.float32, [None, 15])

nb_classes = 2 # 0, 1
#nb_classes = 10 # 0 ~ 9
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 1
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape", Y_one_hot)

print ("X", X)

keep_prob = tf.placeholder(tf.float32)

('one_hot', <tf.Tensor 'one_hot:0' shape=(?, 1, 2) dtype=float32>)
('reshape', <tf.Tensor 'Reshape:0' shape=(?, 2) dtype=float32>)
('X', <tf.Tensor 'Placeholder:0' shape=(?, 15) dtype=float32>)


In [17]:
W1 = tf.Variable(tf.random_normal([15, 9]), name='weight1')
#W21 = tf.Variable(tf.random_normal([9, 5]), name='weight21')
#W22 = tf.Variable(tf.random_normal([5, 5]), name='weight22')
#W2 = tf.Variable(tf.random_normal([5, 5]), name='weight2')
W2 = tf.Variable(tf.random_normal([9, 5]), name='weight2')
W3 = tf.Variable(tf.random_normal([5, nb_classes]), name='weight3')

b1 = tf.Variable(tf.random_normal([9]), name='bias1')
#b21 = tf.Variable(tf.random_normal([5]), name='bias21')
#b22 = tf.Variable(tf.random_normal([5]), name='bias22')
b2 = tf.Variable(tf.random_normal([5]), name='bias2')
b3 = tf.Variable(tf.random_normal([nb_classes]), name='bias3')

with tf.name_scope("layer1") as scope:
    layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)
#with tf.name_scope("layer21") as scope:
#    layer21 = tf.nn.relu(tf.matmul(layer1, W21) + b21)
#    layer21 = tf.nn.dropout(layer21, keep_prob=keep_prob)
#with tf.name_scope("layer22") as scope:
#    layer22 = tf.nn.relu(tf.matmul(layer21, W22) + b22)
#    layer22 = tf.nn.dropout(layer22, keep_prob=keep_prob)

#with tf.name_scope("layer2") as scope:
#    layer2 = tf.nn.relu(tf.matmul(layer22, W2) + b2)
#    layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)
     

with tf.name_scope("layer2") as scope:
    layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
    layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)
    

with tf.name_scope("last") as scope:
    hypothesis = tf.matmul(layer2, W3) + b3

# cost/loss function
#cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
 #                      tf.log(1 - hypothesis))
#train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
     logits=hypothesis, labels=Y_one_hot))
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





In [18]:
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        sess.run(train, feed_dict={X: x_train, Y: y_train, keep_prob: 0.7})
        #sess.run(train, feed_dict={X: x_train, Y: y_train})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_train, Y: y_train, keep_prob: 0.7}))
            #print(step, sess.run(cost, feed_dict={
            #      X: x_data, Y: y_data}))

  
  # Accuracy report
    h, c, a = sess.run([hypothesis, prediction, accuracy],
                       feed_dict={X: x_test, Y: y_test, keep_prob: 1.0})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


(0, 1.1977545)
(100, 0.55777645)
(200, 0.53773683)
(300, 0.51977378)
(400, 0.51090348)
(500, 0.49130836)
(600, 0.47714728)
(700, 0.48259687)
(800, 0.47067192)
(900, 0.47094625)
(1000, 0.47254971)
(1100, 0.47016364)
(1200, 0.46381128)
(1300, 0.46547249)
(1400, 0.46706614)
(1500, 0.46737289)
(1600, 0.46040618)
(1700, 0.4586848)
(1800, 0.46594965)
(1900, 0.45836893)
(2000, 0.46407121)
(2100, 0.4651463)
(2200, 0.45721895)
(2300, 0.45690653)
(2400, 0.44936803)
(2500, 0.45156959)
(2600, 0.45737439)
(2700, 0.4607946)
(2800, 0.4535991)
(2900, 0.45708549)
(3000, 0.45268762)
(3100, 0.46325642)
(3200, 0.4634003)
(3300, 0.45708182)
(3400, 0.45274743)
(3500, 0.45875946)
(3600, 0.45270535)
(3700, 0.45360422)
(3800, 0.45959613)
(3900, 0.45869863)
(4000, 0.45076585)
(4100, 0.45490083)
(4200, 0.45313355)
(4300, 0.4519859)
(4400, 0.4580875)
(4500, 0.45031086)
(4600, 0.45829919)
(4700, 0.4564116)
(4800, 0.45526928)
(4900, 0.45317101)
(5000, 0.45301509)
(5100, 0.45195985)
(5200, 0.45504564)
(5300, 0.45572